In [1]:
pip install telethon pandas requests langdetect


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ModuleNotFoundError:
    pass


In [6]:
import os, json, csv, time, re
import asyncio, requests, pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urlparse
from langdetect import detect, DetectorFactory
DetectorFactory.seed=0
load_dotenv()


False

In [7]:
from telethon import TelegramClient, errors
from telethon.tl.functions.channels import JoinChannelRequest


In [8]:
from telethon.sync import TelegramClient
from getpass import getpass

In [9]:
API_ID  = 24527187                       # int is OK
API_HASH = '8a012769064c0218d779c9e23e45fdb0'
PHONE    = '+18573343025'

import os
os.environ["API_ID"]   = str(API_ID)      # must be strings
os.environ["API_HASH"] = API_HASH
os.environ["PHONE"]    = PHONE


In [10]:
# expects API_ID, API_HASH, and PHONE in the environment
API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")
PHONE = os.getenv("PHONE")
if not all([API_ID, API_HASH, PHONE]):
    raise RuntimeError("API_ID, API_HASH and PHONE environment variables must be set")


In [11]:
PROGRESS_JSON = os.getenv('PROGRESS_JSON', '/content/drive/MyDrive/telegram/progress.json')
RAW_CSV = os.getenv('RAW_CSV', '/content/drive/MyDrive/telegram/messages.csv')
CLEAN_CSV = os.getenv('CLEAN_CSV', '/content/drive/MyDrive/telegram/cleaned.csv')
SESSION_NAME = os.getenv('SESSION_NAME', 'tg_session')

In [12]:
BFS_HOPS         = 2     # how many BFS layers
BFS_PER_CH_LIMIT = 400   # how many msgs to scan for forwards each discovered channel
BFS_MIN_SUBS     = 500   # subscriber threshold
BFS_LANG_SAMPLE  = 15    # how many msgs we sample for English
BFS_EN_RATIO     = 0.5   # 50% must be English
BFS_FWD_MIN      = 1     # min forwards needed to count a channel
BFS_INCLUDE_MENTIONS = False  # parse @username / t.me links
BFS_TOPIC_KEYWORDS = []       # keywords to filter topics
BFS_TOPIC_RATIO   = 0         # ratio threshold for keyword matches
BFS_TOPIC_SAMPLE  = 20        # messages sampled for topic ratio
BFS_MAX_DORMANT_DAYS = 0      # ignore channels dormant this many days
BFS_PAUSE_SEC     = 0         # optional delay between API calls

In [13]:
channels_df = pd.read_csv('telegram_misinformation_channels_full.csv')
def _parse_subs(s):
    s = str(s).replace('+', '').strip()
    mult = 1
    if s.lower().endswith('k'):
        mult = 1000
        s = s[:-1]
    elif s.lower().endswith('m'):
        mult = 1000000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except ValueError:
        return 0
channels_df['subs_num'] = channels_df['Subscribers'].apply(_parse_subs)
seed_usernames = channels_df['Username'].tolist()
FOLLOWERS_DICT = channels_df.set_index('Username')['subs_num'].to_dict()

In [14]:
CHANNELS = seed_usernames

In [15]:
client=TelegramClient(SESSION_NAME,API_ID,API_HASH)
def load_progress():
  if os.path.exists(PROGRESS_JSON):
    with open(PROGRESS_JSON,'r')as f:return json.load(f)
  return {}


In [16]:
def save_progress(d):
  os.makedirs(os.path.dirname(PROGRESS_JSON),exist_ok=True)
  with open(PROGRESS_JSON,'w')as f:json.dump(d,f)
  print("Progress saved.")


In [17]:
def append_header_if_needed():
  ex=os.path.exists(RAW_CSV)
  os.makedirs(os.path.dirname(RAW_CSV),exist_ok=True)
  if not ex:
    with open(RAW_CSV,'w',newline='',encoding='utf-8')as f:
      csv.writer(f).writerow(['channel','msg_id','date','text','fwd_from','fwd_id'])


In [18]:
async def join_if_needed(ent):
  if getattr(ent,'megagroup',False):
    try:await client(JoinChannelRequest(ent))
    except:pass


In [19]:
async def scrape_channel(name,last_id):
  try:ent=await client.get_entity(name)
  except Exception as e:print("Skip",name,e);return last_id
  await join_if_needed(ent);mx=last_id;cnt=0
  async for msg in client.iter_messages(ent,min_id=last_id,reverse=False):
    if BFS_PAUSE_SEC:
      await asyncio.sleep(BFS_PAUSE_SEC)
    cnt+=1;mx=max(mx,msg.id);fw,fwid=None,None
    if msg.forward:
      if msg.forward.chat:fw=msg.forward.chat.title or msg.forward.chat.username;fwid=getattr(msg.forward.chat,'id',None)
      elif msg.forward.sender:fw=msg.forward.sender.first_name or msg.forward.sender.username or'Private';fwid=getattr(msg.forward.sender,'id',None)
    with open(RAW_CSV,'a',newline='',encoding='utf-8')as f:
      csv.writer(f).writerow([name,msg.id,msg.date,msg.text or'',fw,fwid])
  print(name,"+%d msgs"%cnt,"up to",mx);return mx


In [20]:
def run_scrape():
  asyncio.run(scrape_once())
  print("Scraping done.")


In [21]:
def deduplicate(messages):
  out=[];seen=set()
  for m in messages:
    k=(m.get('fwd_id'),m.get('text'))
    if m.get('fwd_id'):
      if k in seen:continue
      seen.add(k)
    out.append(m)
  return out


In [22]:
def unshorten_url(u,t=5):
  try:return requests.head(u,allow_redirects=True,timeout=t).url
  except:return u


In [23]:
def expand_and_extract(m):
  txt=m['text'];urls=re.findall(r'https?://\S+',txt)
  dom=set()
  for u in urls:
    fin=unshorten_url(u);d=urlparse(fin).netloc
    dom.add(d)
    txt=txt.replace(u,fin)
  m['text']=txt;m['domains']=list(dom);return m


In [24]:
def is_english(txt):
  if not txt.strip():return False
  try:return(detect(txt)=='en')
  except:return False


In [25]:
def anonymize(m,user_map):
  if m.get('fwd_id')and m['fwd_from']and not str(m['fwd_from']).startswith(('Channel','@')):
    if m['fwd_id'] not in user_map:user_map[m['fwd_id']]=f"User{len(user_map)+1}"
    m['fwd_from']=user_map[m['fwd_id']];m['fwd_id']=None
  m['text']=re.sub(r'@[\w\d_]+','@USER',m['text'])
  return m


In [26]:
def preprocess_data():
  if not os.path.exists(RAW_CSV):print("No raw CSV.");return
  df=pd.read_csv(RAW_CSV,names=['chan','msg_id','date','text','fwd_from','fwd_id'],header=0)
  msgs=df.to_dict('records');msgs=deduplicate(msgs)
  out=[];umap={}
  for m in msgs:
    m=expand_and_extract(m)
    if is_english(m['text']):m=anonymize(m,umap);out.append(m)
  pd.DataFrame(out).to_csv(CLEAN_CSV,index=False)
  print(f"Saved {len(out)} cleaned msgs to {CLEAN_CSV}")


In [27]:
# Load existing progress data and merge new seeds
progress = load_progress()
for username in seed_usernames:
    if username not in progress:
        progress[username] = 0  # mark as not yet scraped


In [28]:
from pathlib import Path
import os

# Where should we store run-time artefacts?
WORKDIR = Path(os.getenv("TG_WORKDIR", Path.cwd() / "telegram_data"))
WORKDIR.mkdir(parents=True, exist_ok=True)   # makes ./telegram_data if absent

PROGRESS_JSON = WORKDIR / "progress.json"
RAW_CSV       = WORKDIR / "messages.csv"
CLEAN_CSV     = WORKDIR / "cleaned.csv"

In [29]:
# Persist updated progress
save_progress(progress)

Progress saved.


In [30]:
from langdetect import detect
async def _en_ratio(ent, sample=BFS_LANG_SAMPLE):
    en = 0
    async for m in client.iter_messages(ent, limit=sample):
        t = (m.message or '').strip()
        if t:
            try:  en += detect(t)=='en'
            except: pass
    return 0 if not sample else en/sample


In [31]:
from telethon.tl.types import PeerChannel
async def _forward_sources(ent, limit=BFS_PER_CH_LIMIT):
    counts = {}
    async for m in client.iter_messages(ent, limit=limit):
        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)
        fwd = m.forward
        if not fwd:
            continue
        pc = fwd.chat or fwd.original_fwd
        if isinstance(pc, PeerChannel):
            cid = pc.channel_id
            counts[cid] = counts.get(cid, 0) + 1
    return {c for c, n in counts.items() if n >= BFS_FWD_MIN}
MENTION_RE = re.compile(r'@([\w\d_]{5,32})|t\.me/([\w\d_]{5,32})', re.I)
async def _mention_sources(ent, limit=BFS_PER_CH_LIMIT):
    src = set()
    if not BFS_INCLUDE_MENTIONS:
        return src
    async for m in client.iter_messages(ent, limit=limit):
        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)
        text = (m.message or '')
        for a, b in MENTION_RE.findall(text):
            name = a or b
            if name:
                src.add(name.lstrip('@'))
    return src
async def _topic_ratio(ent, keywords, sample=BFS_TOPIC_SAMPLE):
    if not keywords:
        return 1.0
    pat = re.compile('|'.join(re.escape(k) for k in keywords), re.I)
    hits = 0
    tot = 0
    async for m in client.iter_messages(ent, limit=sample):
        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)
        text = (m.message or '')
        if text.strip():
            tot += 1
            if pat.search(text):
                hits += 1
    return 0 if not tot else hits / tot


In [32]:
async def _is_valid(ent):
    username = (getattr(ent, 'username', '') or '').strip()
    count = FOLLOWERS_DICT.get(username, 0)
    if count == 0:
        try:
            full = await client(GetFullChannelRequest(ent))
            count = getattr(full.full_chat, 'participants_count', 0)
        except Exception:
            count = 0
    if count < BFS_MIN_SUBS:
        return False
    if BFS_MAX_DORMANT_DAYS:
        try:
            async for m in client.iter_messages(ent, limit=1):
                if BFS_PAUSE_SEC:
                    await asyncio.sleep(BFS_PAUSE_SEC)
                if m.date:
                    days = (datetime.utcnow() - m.date.replace(tzinfo=None)).days
                    if days > BFS_MAX_DORMANT_DAYS:
                        return False
                break
        except Exception:
            return False
    if BFS_TOPIC_KEYWORDS and BFS_TOPIC_RATIO:
        try:
            if await _topic_ratio(ent, BFS_TOPIC_KEYWORDS, BFS_TOPIC_SAMPLE) < BFS_TOPIC_RATIO:
                return False
        except Exception:
            return False
    try:
        return (await _en_ratio(ent)) >= BFS_EN_RATIO
    except Exception:
        return False


In [33]:
async def snowball_discovery():
    global CHANNELS
    progress = load_progress()
    seeds = [c.lstrip('@') for c in CHANNELS]
    validated = await discover_bfs(seeds, max_hops=BFS_HOPS)
    for v in validated:
        if v not in progress:
            progress[v] = 0
    save_progress(progress)
    CHANNELS = list(set(CHANNELS) | set(validated))
    print('validated:', len(validated), '| total CHANNELS:', len(CHANNELS))


In [34]:
async def scrape_once():
    await client.start(phone=PHONE)
    await snowball_discovery()          # ①  discover / merge
    last = load_progress(); append_header_if_needed()
    for c in CHANNELS:
        last[c] = await scrape_channel(c, last.get(c,0))
        if BFS_PAUSE_SEC:
            await asyncio.sleep(BFS_PAUSE_SEC)
    save_progress(last); await client.disconnect()


In [37]:
def main():
  run_scrape()
  preprocess_data()
  print("Done.")


In [ ]:
# Enhanced BFS crawler with forward counting and optional mentions
from collections import deque, defaultdict

async def discover_bfs(seeds, max_hops=BFS_HOPS):
    visited = set(seeds)
    queue = deque((s, 0) for s in seeds)
    validated = set(seeds)
    while queue:
        username, hop = queue.popleft()
        if hop >= max_hops:
            continue
        try:
            ent = await client.get_entity(username)
        except Exception:
            continue
        if not await _is_valid(ent):
            continue
        fwd = await _forward_sources(ent)
        mentions = await _mention_sources(ent)
        for cid in fwd:
            try:
                ch = await client.get_entity(PeerChannel(cid))
                uname = ch.username
                if uname and uname not in visited:
                    visited.add(uname)
                    queue.append((uname, hop + 1))
                    validated.add(uname)
            except Exception:
                pass
            if BFS_PAUSE_SEC:
                await asyncio.sleep(BFS_PAUSE_SEC)
        for uname in mentions:
            if uname not in visited:
                visited.add(uname)
                queue.append((uname, hop + 1))
                validated.add(uname)
            if BFS_PAUSE_SEC:
                await asyncio.sleep(BFS_PAUSE_SEC)
    return validated


In [36]:
# main()